<a href="https://colab.research.google.com/github/ha-pu/data_course/blob/Google_Colab/3-regressions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

+ title: Regression Analysis
+ author: Harald Puhr
+ date: July 20, 2025

[Data Source](https://github.com/ha-pu/data_files/blob/main/README.md#car-evaluation-dataset)

# Load tidyverse package and data

In [ ]:
install.packages(c("sjPlot", "tidverse"))

In [ ]:
library(sjPlot)
library(tidyverse)

In [ ]:
df <- read_csv("https://raw.githubusercontent.com/ha-pu/data_files/refs/heads/main/2-car_acceptance.csv")

Code variables `lug_boot`, `safety`, `doors`, and `persons` as type factor to
better handle categorical data.

In [ ]:
df$lug_boot <- factor(df$lug_boot, levels = c("FALSE", "TRUE"))
df$safety <- factor(df$safety, levels = c("low", "med", "high"))
df$doors <- factor(df$doors, levels = c("2", "3", "4", "5more"))
df$persons <- factor(df$persons, levels = c("2", "4", "more"))

# Basic data exploration

In [ ]:
slice_head(df)

In [ ]:
glimpse(df)

In [ ]:
summary(df)

In [ ]:
df %>%
    ggplot() +
    geom_histogram(aes(x = class), bins = 25) +
    theme_bw()

In [ ]:
df %>%
    select(where(is.numeric)) %>%
    cor()

# Intercept-only model

## Fit model

In [ ]:
mod1 <- lm(class ~ 1, data = df)

In [ ]:
summary(mod1)

The function `tab_model` from the package sjPlot returns similar information as
`summary` but formats it more nicely. In this script, we will continue working
with `tab_model` instead of `summary` to analyze our models.

In [ ]:
tab_model(mod1)

## Questions

+ Does the intercept-only model tell us anything about the data?
+ Would you rely on this model alone for anything actionable?
+ What does this model actually predict?

# Direct effects model

## Fit model

In [ ]:
mod2 <- lm(class ~ doors + persons + lug_boot + safety, data = df)
mod3 <- lm(class ~ buying + maint, data = df)
mod4 <- lm(class ~ doors + persons + lug_boot + safety + buying + maint, data = df)

In [ ]:
tab_model(mod2, mod3, mod4)

In [ ]:
tibble(
    actual = df$class,
    mod2 = predict(mod2),
    mod3 = predict(mod3),
    mod4 = predict(mod4)
) %>%
    pivot_longer(cols = -actual) %>%
    ggplot() +
    geom_point(aes(x = actual, y = value, colour = name)) +
    geom_smooth(aes(x = actual, y = value, colour = name), method = "lm", se = FALSE) +
    geom_abline(slope = 1, intercept = 0) +
    labs(
        x = "Actual class",
        y = "Predicted class",
        colour = NULL
    ) +
    theme_bw()

## Questions

+ What should the product managers do to increase the car acceptance?
+ How does a discount of 5,000 USD on the price affect the car acceptance?
+ Which features have the strongest relationship with acceptance? Why might that
  be the case?
+ Why might the number of doors have a small effect on class?
+ Model 4 has the highest R-squared. Why does adding price-related features
  improve it?

# Interaction metric x metric

We can add an interaction term, a multiplicative relationship between two
variables, by adding a term `var1:var2` to the regression equation.

## Fit model

In [ ]:
mod5 <- lm(class ~ doors + persons + lug_boot + safety + buying + maint + buying:maint, data = df)

In [ ]:
tab_model(mod5)

In [ ]:
plot_model(mod5, type = "pred", terms = c("buying", "maint"))

## Questions

+ Should your marketing department offer a discount on maintenance for new
  customers?
+ How might the effect of buying on car acceptance depend on the level of
  maintenance?
+ Is the direction of the interaction consistent with what you might expect in
  real life?
+ Do you think a simple additive model is enough for understanding how
  price-related factors influence car acceptance?

# Interaction metric x factor

## Fit model

In [ ]:
mod6 <- lm(class ~ doors + persons + lug_boot + safety + buying + maint + buying:lug_boot, data = df)
mod7 <- lm(class ~ doors + persons + lug_boot + safety + buying + maint + buying:safety, data = df)

In [ ]:
tab_model(mod6)

In [ ]:
plot_model(mod6, type = "pred", terms = c("buying", "lug_boot"))

In [ ]:
tab_model(mod7)

In [ ]:
plot_model(mod7, type = "pred", terms = c("buying", "safety"))

## Questions

+ Does greater storage space justify a higher price?
+ The interaction term `buying:lug_bootTRUE` is negative and significant. What
  does this tell us?
+ Look at the main effect of `lug_bootTRUE` and the interaction. How do these
  work together?
+ Why might the benefit of a large luggage space decrease as a car becomes more
  expensive?
+ Does this interaction feel intuitive in a real-world consumer context? If not,
  what might explain it?
+ Does a higher safety level justify a higher price?
+ Why might higher-priced safe cars get lower class scores?
+ Do you think the price of a car should influence class more or less when the
  car is very safe? Why?

# Interaction factor x factor

## Fit model

In [ ]:
mod8 <- lm(class ~ doors + persons + lug_boot + safety + buying + maint + lug_boot:safety, data = df)

In [ ]:
tab_model(mod8)

In [ ]:
plot_model(mod8, type = "pred", terms = c("lug_boot", "safety"))

## Questions

+ Does greater storage space balance lower safety levels?
+ The main effect of `lug_bootTRUE` is not significant, but both interactions
  with safety are highly significant. What does this imply?
+ How do these numbers change the story about the value of trunk space?
+ Imagine two cars with the same features, except one has a big trunk. When
  would that big trunk boost the car's predicted class score?
+ Why might a large trunk be more valued when the car is also moderately or
  highly safe?
+ Does the model suggest that trunk size matters more for safer vehicles?

# Logistic regression

## Transform binary variable from factor to logical

We had to transform the `lug_boot` variable to factor for some of the functions
used above. Now we will transform it back to a logical variable for the logistic
regression.


In [ ]:
df$lug_boot <- df$lug_boot == "TRUE"

## Fit model

In [ ]:
mod9 <- glm(lug_boot ~ class + doors + safety + buying + maint, data = df, family = binomial(link = "logit"))

In [ ]:
tab_model(mod9, transform = NULL)

In [ ]:
tab_model(mod9)